# Applying Machine Learning to Sentiment Analysis

## Chapter 8 from Sebastian Raschka's Python Machine Learning

Sentiment Analysis is a sub-discipline of **Natural Language Proprocessing (NLP)**.  

We will be classifying documents based on their polarity: the attitude of the writer.

We will be using a dataset that consists of 25,000 positive movie reviews and 25,000 negative movie reviews for a total of 50,000 reviews that will be split into training and testing sets.

The movie reviews come from the **Internet Movie Database (IMDB)**.  

The purpose is to build a model that can accurately predict the sentiment of a movie review on new data.

The dataset was specifically obtained from [http://ai.stanford.edu/~amaas/data/sentiment/]

In [3]:
import pandas as pd
import numpy as np

# Here I'm using an the same dataset on the Stanford site but previously downloaded and shuffled in a random way
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head()

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


In [4]:
df.shape

(50000, 2)

In [5]:
df.dtypes

review       object
sentiment     int64
dtype: object

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
review       50000 non-null object
sentiment    50000 non-null int64
dtypes: int64(1), object(1)
memory usage: 781.3+ KB


In [7]:
df.isna().sum()

review       0
sentiment    0
dtype: int64

In [8]:
df['sentiment'].value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

For df[sentiment] 1 = positive review, 0 = negative review

## Introducing the bag-of-words model

**Bag-of-Word** method allows us to represent textual data as numerical vectors.

Specifically,
1. We create a set of tokens - words - from the entire set of documents
2. We construct a feature vector from each document that counts the frequency of each word in a particular document.

### Transforming words into feature vectors

We can use the **CountVectorizer** class from scikit-learn to help us out.  **CountVectorizer** takes an array of text data, sentences or entire documents, and turns it into a bag-of-words model: